# Boots 'n' Cats 2a: Modelling with Rekognition Custom Labels

In this notebook we'll use [Amazon Rekognition](https://aws.amazon.com/rekognition/)'s new [custom labels](https://aws.amazon.com/rekognition/custom-labels-features/) functionality (announced at Re:Invent 2019) to train a boots 'n' cats detector with minimal data science knowledge required.

You'll need to have gone through the first notebook in this series (*Intro and Data Preparation*) to complete this example.

## Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs.

For this particular notebook, we'll also **update** the core AWS libraries because Rekognition Custom Labels are so new!

In [ ]:
!pip install --upgrade awscli boto3

In [ ]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import os

# External Dependencies:
import boto3
import sagemaker
from IPython.display import display, HTML

# Local Dependencies:
%aimport util

Next we re-load configuration from the intro & data processing notebook:

In [ ]:
%store -r BUCKET_NAME
assert BUCKET_NAME, "BUCKET_NAME missing from IPython store"
%store -r CLASS_NAMES
assert CLASS_NAMES, "CLASS_NAMES missing from IPython store"
%store -r test_image_folder
assert test_image_folder, "test_image_folder missing from IPython store"

Now we connect to the AWS SDKs we'll use, and validate the choice of S3 bucket:

**If you get a Rekognition permissions error:** The configured IAM Role for this SageMaker notebook instance doesn't have access to Rekognition.

* Go to the notebook instance in the SageMaker console (*Notebook > Notebook Instances > Click on this notebook's name*)
* Scroll down to *Permissions and encryption* and click on the hyperlinked "IAM Role ARN"
* "Attach Policies" for *AmazonRekognitionFullAccess*

In [ ]:
role = sagemaker.get_execution_role()
session = boto3.session.Session()
region = session.region_name
rekognition = session.client("rekognition")

bucket_region = \
    session.client("s3").head_bucket(Bucket=BUCKET_NAME)["ResponseMetadata"]["HTTPHeaders"]["x-amz-bucket-region"]
assert (
    bucket_region == region
), f"Your S3 bucket {BUCKET_NAME} and this notebook need to be in the same region."

if (region != "us-east-1"):
    print("WARNING: Rekognition Custom Labels functionality is only available in us-east-1 at launch")

## Step 1: Create the Rekognition Custom Labels project

In the Rekognition Console (*Services > Amazon Rekognition* in the AWS console):

* Select **Custom Labels** from the Rekognition sidebar menu (If you don't see this menu item, you might be in a region where Rekognition Custom Labels functionality is not yet available!)
* Click the **Get started** button from the dashboard, or **Projects > Create project** from the expandable Rekognition Custom Labels sidebar menu
* **Name** your project `bootsncats` and click create

You should be forwarded to the project overview screen for your new, empty project.

Note that, as usual in AWS, these console entities can also be queried via the API. Check the `project_name` below matches what you created:

In [ ]:
project_name = # TODO: Whatever you entered in the console! "bootsncats"?

# Look up the project ARN from the project name:
try:
    project_description = next(filter(
        lambda p: p["ProjectArn"].partition("/")[2].partition("/")[0] == project_name,
        rekognition.describe_projects()["ProjectDescriptions"]
    ))
    print(project_description)
    project_arn = project_description["ProjectArn"]
except StopIteration as e:
    raise ValueError(
        f"No project found matching name '{project_name}'"
    ) from e

## Step 2: Create training and validation datasets

From the *Datasets* section, click the **Create dataset** button:

* We'll name our training dataset `boots-and-cats`
* Select the first type *"Import images labeled by SageMaker Ground Truth"*
* For the **manifest location**, select the **training manifest S3 URI** as printed near the end of the intro and data preparation notebook

<img src="BlogImages/CreateRekognitionDataset.png"/>

**Repeat** this process for the validation data-set, this time naming the Rekognition dataset `boots-and-cats-validation`, and using the **validation manifest S3 URI**

## Step 3: Train the Rekognition Custom Labels Model

Now that both datasets are created, you can train a model.

The project ARN should be pre-populated if you clicked *"Train model"* from the project details page; but if not you can find it from the output of Step 1 in this notebook.

Select `boots-and-cats` for the training dataset, and `boots-and-cats-validation` for the test dataset, and click **Train** to get started.

<img src="BlogImages/RekognitionTrainModel.png"/>

## Step 4: While the model trains...

Training the model can take several minutes, so why not use this time to start looking at the next notebook with a different modelling approach - SageMaker built-in algorithms?

You can check up on the model training process in the AWS Console (you might need to refresh the page for latest updates), or by running the snippet below:

In [ ]:
print("\n".join(list(map(
    lambda d: f'{d["Status"]}: {d["ProjectVersionArn"]}',
    rekognition.describe_project_versions(
        ProjectArn=project_arn
    )["ProjectVersionDescriptions"]
))))

We'll need the **model ARN** to call the model in the next step. You can fetch it either from the output above, or through the Console UI. (But the Console UI currently only shows the model ARN *after* training is complete - by clicking on the hyperlinked model name in the project details page)

In [ ]:
model_arn = # TODO: From above

# Check the status to validate the above:
try:
    project_version_description = next(filter(
        lambda v: v["ProjectVersionArn"] == model_arn,
        rekognition.describe_project_versions(
            ProjectArn=project_arn
        )["ProjectVersionDescriptions"]
    ))
    print(project_version_description["Status"])
except StopIteration as e:
    raise ValueError(
        f"No project version (model) ARN found matching:\n{model_arn}"
    ) from e

## Step 5: Review model metrics

When the model training is complete, you should be able to click on the hyperlinked model name from the project details page, to access the model overview:

<img src="BlogImages/RekognitionModelDetails.png"/>

A range of classification-oriented statistics ([precision, recall, and F1 score](https://en.wikipedia.org/wiki/Precision_and_recall)) are presented to help you evaluate the quality of the trained model versus the provided test/validation set and decide whether to deploy it: Including both summary/overall and per-class figures.

Rekognition also calculates an **Assumed threshold** for each class: the "optimal" confidence score threshold detections must pass to be reported in the output.

However, simply maximising the F1 score for this "optimum" isn't right for every use case: E.g. in fraud detection or medical disease detection the value of a false positive might be very different from a false negative. For this reason you can still override the threshold when making API requests

## Step 6: Deploy the model

[Pricing](https://aws.amazon.com/rekognition/pricing/) for Rekognition Custom Labels inference is by **capacity**: We deploy ("start") our model with a specified capacity which translates to transactions-per-second [as documented here](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/rm-run-model.html).

Note that the model may take a minute or two to start and stop. You can re-run the code from **Step 4** to check up on the status of your models.

In [ ]:
rekognition.start_project_version(
    ProjectVersionArn=model_arn,
    MinInferenceUnits=1
)

## Step 7: Use the model

When the model is `RUNNING`, we can call it via API to perform inference on our test images as below.

As discussed before, we can **override the *AssumedConfidence*** threshold selected by Rekognition - or even request **all** detections and filter the result with a `confidence_threshold` in post-processing:

In [ ]:
# Set to "None" to use Rekognition's default, or else a float in 0-100
# (Yes it's weird that Rekognition wants a pctage not a 0-1...)
override_confidence = # TODO: Try None (no quotes), 80, and some other numbers

for test_image in os.listdir(test_image_folder):
    test_image_path = f"{test_image_folder}/{test_image}"
    with open(test_image_path, "rb") as f:
        payload = bytearray(f.read())
        
    response = rekognition.detect_custom_labels(
        ProjectVersionArn=model_arn,
        Image={ "Bytes": payload },
        MinConfidence=override_confidence
    )

    result = list(map(
        lambda l: [
            CLASS_NAMES.index(l["Name"]),  # Class ID
            l["Confidence"],  # Confidence score
            l["Geometry"]["BoundingBox"]["Left"],  # x1
            l["Geometry"]["BoundingBox"]["Top"],  # y1
            l["Geometry"]["BoundingBox"]["Left"]
            + l["Geometry"]["BoundingBox"]["Width"],  # x2
            l["Geometry"]["BoundingBox"]["Top"]
            + l["Geometry"]["BoundingBox"]["Height"],  # y2
        ],
        response["CustomLabels"]
    ))
    # result is a list of [class_ix, confidence, x1, y1, x2, y2] detections.
    display(HTML(f"<h4>{test_image}</h4>"))
    util.visualize_detection(
        test_image_path,
        result,
        CLASS_NAMES,
        thresh=0 # No cutoff needed since Rekognition is already filtering for us
    )

## Clean up

When we're done with our model, we should *stop* it to avoid any ongoing charges for inference capacity:

In [ ]:
rekognition.stop_project_version(
    ProjectVersionArn=model_arn
)

## Review

Hopefully you found it easy to create a Rekognition Custom Labels model deployment from our SageMaker Ground Truth labelled data: It's a great entry point for standard computer vision tasks without needing deep neural network skills, and you might find in our other notebooks that the automatic learning out-performs basic custom models unless they're very carefully optimized!

Remember that the "Manifest files" we used as inputs for this model training were just JSONLines documents with a particular schema, and we actually already modified them in the data preparation notebook to standardize between batches: If you have image annotations in a different format, it's pretty easy to convert them for easy ingestion.

Thanks for taking the time to explore this notebook and the others in the series: We'd love to hear your feedback!